In [33]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [4]:
# !pip3 install pymongo
# !pip3 install mongoengine
# !pip3 install Faker

<IPython.core.display.Javascript object>

In [17]:
# Prepare environment for importing from src
import sys

sys.path.insert(0, "..")

<IPython.core.display.Javascript object>

## Create Some Fake Data! 

In [28]:
import random

from faker import Faker
from src.data import drug_formulas, specialties, drug_names

fake = Faker()

<IPython.core.display.Javascript object>

In [24]:
def generate_national_id(prev_national_ids):
    nat_id = fake.numerify(text="##########")
    while nat_id in prev_national_ids:
        nat_id = fake.numerify(text="##########")
    prev_national_ids.add(nat_id)
    return nat_id

<IPython.core.display.Javascript object>

In [72]:
docs_national_ids = set()
patients_national_ids = set()
doctors = []
patients = []
pharmacies = []
companies = []
contracts = []
drugs = []
prescription_dates = []
for _ in range(10):
    doctors.append(
        {
            "national_id": generate_national_id(docs_national_ids),
            "first_name": fake.first_name(),
            "last_name": fake.last_name(),
            "specialty": random.choice(specialties),
            "backgronud": random.randint(0, 80),
        }
    )
    patients.append(
        {
            "national_id": generate_national_id(patients_national_ids),
            "first_name": fake.first_name(),
            "last_name": fake.last_name(),
            "address": fake.address(),
            "birthdate": str(fake.profile(fields=["birthdate"])["birthdate"]),
            "password": fake.profile(fields=["username"])["username"],
            "main_doctor": random.sample(docs_national_ids, 1)[0],
        }
    )
    pharmacies.append(
        {
            "name": fake.company(),
            "address": fake.address(),
            "telephone": fake.phone_number(),
        }
    )
    companies.append(
        {
            "name": fake.company(),
            "telephone": fake.phone_number(),
        }
    )
    contracts.append(
        {
            "text": fake.paragraph(nb_sentences=5),
            "start_date": str(
                fake.date_this_decade(before_today=True, after_today=False)
            ),
            "end_date": str(fake.future_date()),
        }
    )
    drugs.append(
        {
            "name": random.choice(drug_names),
            "formula": random.choice(drug_formulas),
        }
    )
data = {
    "doctors": doctors,
    "patients": patients,
    "pharmacies": pharmacies,
    "companies": companies,
    "contracts": contracts,
    "drugs": drugs,
    "prescription_dates": prescription_dates,
}

<IPython.core.display.Javascript object>

In [73]:
import json

with open("medical_data.json", "w") as json_file:
    json.dump(data, json_file)

<IPython.core.display.Javascript object>

## Define Documents' Schema 

In [91]:
class Doctor(Document):
    national_id = StringField(
        min_length=10, max_length=10, required=True, primary_key=True
    )
    first_name = StringField(max_length=50, required=True)
    last_name = StringField(max_length=50, required=True)
    specialty = StringField(max_length=50, required=True)
    background = IntField(min_value=0, max_value=100, required=True)


class Patient(Document):
    national_id = StringField(
        min_length=10, max_length=10, required=True, primary_key=True
    )
    first_name = StringField(max_length=50, required=True)
    last_name = StringField(max_length=50, required=True)
    address = StringField(max_length=256)
    birthdate = DateField(required=True)
    password = StringField(max_length=256, required=True)
    main_doctor = ReferenceField(Doctor, required=True)


class Pharmacy(Document):
    name = StringField(max_length=256, required=True)
    address = StringField(max_length=256)
    telephone = StringField(max_length=256)


class Company(Document):
    name = StringField(max_length=50, required=True, primary_key=True)
    telephone = StringField(max_length=15)


class Contract(Document):
    text = StringField()
    start_date = DateField(required=True)
    end_date = DateField(required=True)
    pharmacy = ReferenceField(Pharmacy, required=True)
    company = ReferenceField(Company, required=True)


class Drug(Document):
    name = StringField(
        max_length=50,
        required=True,
        unique_with="company",
    )
    company = ReferenceField(Company, required=True)
    formula = StringField(max_length=100, required=True)


class Sale(Document):
    drug = ReferenceField(Drug, required=True, unique_with="store")
    store = ReferenceField(Pharmacy, required=True)
    price = FloatField(required=True)


class PrescriptionItem(EmbeddedDocument):
    drugs = ReferenceField(Drug, required=True)
    quantity = IntField(min_value=0, required=True)


class Prescription(Document):
    date = DateField(required=True)
    items = EmbeddedDocumentListField(PrescriptionItem, required=True)
    doctor = ReferenceField(Doctor, required=True)
    patient = ReferenceField(Patient, required=True)

<IPython.core.display.Javascript object>

In [95]:
from bson.objectid import ObjectId

TYPE = (
    ("C", "Children"),
    ("M", "Men"),
    ("W", "Women"),
)


class Product(Document):
    name = StringField(max_length=128, required=True, unique_with=["seller", "ptype"])
    seller = StringField(max_length=128, required=True)
    ptype = StringField(max_length=1, choices=TYPE)
    price = FloatField(min_value=0.0)


SIZE = (
    ("S", "Small"),
    ("M", "Medium"),
    ("L", "Large"),
    ("XL", "Extra Large"),
    ("XXL", "Extra Extra Large"),
)


class ProductItem(Document):
    #     https://stackoverflow.com/questions/10144852/how-can-i-create-unique-ids-for-embedded-documents-in-mongodb
    #     _id = ObjectIdField( required=True, default=lambda: ObjectId() )
    product = ReferenceField(
        Product, reverse_delete_rule=CASCADE, unique_with=["size", "color"]
    )
    color = StringField(max_length=64, required=True)
    size = StringField(max_length=3, choices=SIZE, required=True)
    quantity = IntField(min_value=0, required=True)


class BasketItem(EmbeddedDocument):
    product_item = ReferenceField(ProductItem)
    quantity = IntField(min_value=1, default=1, required=True)


class User(Document):
    email = EmailField(required=True, primary_key=True)
    first_name = StringField(max_length=50, required=True)
    last_name = StringField(max_length=50, required=True)
    city = StringField(max_length=50)
    address = StringField(max_length=256)
    telephone = StringField(max_length=15)
    password = StringField(max_length=256, required=True)
    basket_items = EmbeddedDocumentListField(BasketItem)


class Driver(Document):
    national_id = StringField(
        min_length=10, max_length=10, required=True, primary_key=True
    )
    first_name = StringField(max_length=50, required=True)
    last_name = StringField(max_length=50, required=True)
    telephone = StringField(max_length=15)
    plate_number = StringField(max_length=15)


DELIVERY_TYPES = (
    ("N", "Normal"),
    ("S", "Special"),
)
DELIVERY_STATUSES = (("P", "Pending"), ("R", "Registered"), ("C", "Completed"))


class Order(Document):
    user = ReferenceField(document_type=User, reverse_delete_rule=NULLIFY)
    date = DateField(required=True)
    items = EmbeddedDocumentListField(BasketItem)
    delivery_type = StringField(max_length=1, choices=DELIVERY_TYPES, required=True)
    delivery_status = StringField(
        max_length=1, choices=DELIVERY_STATUSES, required=True
    )


class Delivery(Document):
    date = DateField(required=True)
    capacity = IntField(required=True)
    driver = ReferenceField(Driver, reverse_delete_rule=NULLIFY)


class Comment(Document):
    user = ReferenceField(document_type=User, reverse_delete_rule=NULLIFY)
    product = ReferenceField(Product, reverse_delete_rule=CASCADE)
    rating = IntField(min_value=0, max_value=5, required=True)
    text = StringField()

<IPython.core.display.Javascript object>

In [ ]:
datetime.datetime.strptime("2020-11-24", "%Y-%m-%d").date()